In [1]:
import torch
import time
from dgl.sampling import sample_neighbors
from utils.utils import get_process_graph
from pyvis.network import Network

In [2]:
graph,features,_ = get_process_graph('ogbn-arxiv', -1)

batch = torch.arange(graph.num_nodes())

169343 1166243


In [3]:
def sample(graph, batch_in):
    layers = []
    blocks = []
    # Note. Dont forget self loops otherwise GAT doesnt work.
    # Create bipartite graphs for the first l-1 layers
    last_layer = batch_in
    last_layer = last_layer.unique()
    layers.append(last_layer)
    iterations = len(batch_in)
    fanout_factor = 1.5
    for i in range(iterations):
        fanout = int(7)
        print(fanout,i,len(last_layer))
        dest_ids,src_ids = sample_neighbors(graph, last_layer, fanout).edges()
        self_loop_dests = torch.cat([last_layer, dest_ids])
        edges = self_loop_dests, torch.cat([last_layer, src_ids])
        last_layer = torch.unique(self_loop_dests)
        layers.append(last_layer)
        blocks.append(edges)
    return layers,blocks

In [9]:
num_graphs = 4

layers,blocks = sample(graph, batch[0:num_graphs])

for offset in range(0,num_graphs):

  l1 = layers[offset].tolist()
  l2 = layers[offset+1].tolist()
  b2,b1 = map(lambda x: x.tolist(),blocks[offset])

  l1Max = max(l1)+1
  l2Added = [x + l1Max for x in l2]
  b2Added = [x + l1Max for x in b2]

  l1Label = [str(x) for x in l1]
  l2Label = [str(x) for x in l2]

  colors = ['red','blue','green','purple']

  l1Colors = [colors[features[x]] for x in l1]
  l2Colors = [colors[features[x]] for x in l2]

  net = Network(notebook=True, directed=True)
  
  net.add_nodes(l1,label=l1Label,color=l1Colors,shape=['triangle']*len(l1))
  net.add_nodes(l2Added,label=l2Label,color=l2Colors)
    
  net.add_edges(list(zip(b1,b2Added)))
  
  # net.toggle_physics(False)

  net.show("graphs/graph {}.html".format(offset))

7 0 4
7 1 19
7 2 85
7 3 242
